In [38]:
import pdb
import pandas as pd
pd.options.display.float_format = '{:,.2f}'.format

import numpy as np

import io
import os
import json

# https://github.com/bolaurent/zuora_restful_python
from zuora_restful_python.zuora import Zuora

ZUORA_CONFIG_SANDBOX = json.load(open(os.path.expanduser('~') + '/.zuora-sandbox-config.json'))


zuora_sandbox = Zuora(ZUORA_CONFIG_SANDBOX['user'],\
                      ZUORA_CONFIG_SANDBOX['password'],\
                      endpoint='sandbox')


FIELDS = [
    'RatePlanCharge.DTCV'
    ,'RecognizedRevenueAccountingCode.Name'
    ,'Subscription.Name'
    ,'Subscription.SubscriptionStartDate'
]

# given a date like '2017-01-02', return a period name like '2017-01-01'
def convert_date_to_periodname(d):
    return '{}-{}'.format(d[0:4],d[5:7])

# query rateplancharges and return a pandas dataframe, indexed on subscription name
def read_rateplancharges(zuora):
    q = """
        SELECT {} FROM RatePlanCharge 
        WHERE Subscription.SubscriptionStartDate >= '2017-04-01'
        AND Subscription.SubscriptionStartDate < '2017-07-01'
        """.format(','.join(FIELDS))
    csvData = zuora.query_export(q)
    df = pd.read_csv(io.StringIO(csvData), index_col=['Subscription.Name'])
    
    return df

# query rateplancharges from zuora
rateplancharges = read_rateplancharges(zuora_sandbox)


# create a column that shows the fiscal period, based on the subscription start date
rateplancharges['Month'] = rateplancharges['Subscription.SubscriptionStartDate']\
                            .apply(convert_date_to_periodname)

# pivot the table to show total revenue by account and period
rateplancharges = pd.pivot_table(rateplancharges, \
                                 index=['RecognizedRevenueAccountingCode.Name'], \
                                 columns=['Month'], \
                                 aggfunc=np.sum, \
                                 values=['RatePlanCharge.DTCV'])

# Obfuscate the data for publication
rateplancharges = rateplancharges.applymap(lambda x: np.random.rand() * 100000000)


# print it in a nice table
rateplancharges




RatePlanCharge.DTCV                \
Month                                            2017-04       2017-05   
RecognizedRevenueAccountingCode.Name                                     
Incidental Revenue                         80,628,679.28 51,509,776.43   
Licensing Revenue                          66,375,074.07 42,462,912.07   
Marketplace Revenue                        51,577,078.03 51,623,284.31   
Operations Revenue                         40,145,278.14 40,221,235.34   
Professional Services Revenue               4,831,822.67  7,745,958.48   
Support Revenue                            60,692,366.88 83,165,920.61   

                                                    
Month                                      2017-06  
RecognizedRevenueAccountingCode.Name                
Incidental Revenue                   59,587,133.70  
Licensing Revenue                    48,230,271.30  
Marketplace Revenue                  56,626,367.85  
Operations Revenue                   81,791,669.20  
Professional Services Revenue        27,595,232.64  
Support Revenue                      36,005,086.00